In [22]:
import cooler
import numpy as np
import gzip

# Define paths to your pairix .pairs file and the desired output .cool file
pairs_file = "/Users/School/Desktop/Singh_Repositories/scGrapHic/GSM6998595_GasaE751001.pairs.gz"
cool_file = "/Users/School/Desktop/Singh_Repositories/scGrapHic/GasaE751001.cool"

# Define bin size and chromosome sizes
bin_size = 1000000  # 1 Mb resolution
chromosome_sizes = {
    "chr1":195471971,
    "chr2":182113224,
    "chr3":160039680,
    "chr4":156508116,
    "chr5":151834684,
    "chr6":149736546,
    "chr7":145441459,
    "chr8":129401213,
    "chr9":124595110,
    "chr10":130694993,
    "chr11":122082543,
    "chr12":120129022,
    "chr13":120421639,
    "chr14":124902244,
    "chr15":104043685,
    "chr16":98207768,
    "chr17":94987271,
    "chr18":90702639,
    "chr19":61431566,
    "chrX":171031299
}


# Calculate bins manually
bins = {}
for chrom, size in chromosome_sizes.items():
    num_bins = int(np.ceil(size / bin_size))
    bins[chrom] = np.linspace(0, size, num=num_bins+1, endpoint=True, dtype=int)

# Create an empty dictionary to store pixel data
pixels = {"bin1_id": [], "bin2_id": [], "count": []}

# Read .pairs file and populate the pixel data
with gzip.open(pairs_file, 'rt') as f:  # 'rt' for read in text mode after gzip decompression
    for line in f:
        try:
            parts = line.strip().split('\t')
            chrom1, pos1, _, chrom2, pos2, _ = parts[1:7]
            pos1, pos2 = int(pos1), int(pos2)
            bin1, bin2 = np.digitize([pos1, pos2], bins[chrom1]), np.digitize([pos2, pos2], bins[chrom2])
            # Subtract 1 to convert to 0-based indexing
            bin1 -= 1
            bin2 -= 1
            pixels["bin1_id"].append(bin1)
            pixels["bin2_id"].append(bin2)
            pixels["count"].append(1)  # You can modify this if your .pairs file contains count information
        except ValueError:
            # Skip lines that do not have the expected number of values
            pass


# Create a sparse matrix using the pixel data
pixels["bin1_id"] = np.array(pixels["bin1_id"])
pixels["bin2_id"] = np.array(pixels["bin2_id"])
pixels["count"] = np.array(pixels["count"])
sp_matrix = np.array((pixels["count"], (pixels["bin1_id"], pixels["bin2_id"])))

# Create a Cooler file with the specified binning scheme
cooler.create_cooler(cool_file, bins=bins, assembly="Gasa_E751001", pixels=sp_matrix)

# Access the created .cool file
c = cooler.Cooler(cool_file)

# Now you can work with the Cooler file 'c' for further analysis
# For example, accessing the contact matrix for a specific chromosome
chromosome_name = "chr1"
contact_matrix = c.matrix(balance=True).fetch(chromosome_name)

# Perform your analysis with the contact matrix as needed


/var/folders/mj/5v8fsn3x5q14pn7b_5d48j_r0000gq/T/ipykernel_20722/1112084484.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sp_matrix = np.array((pixels["count"], (pixels["bin1_id"], pixels["bin2_id"])))


KeyError: 'chrom'